In [1]:
import os
from astropy.table import Table
import pickle

In [16]:
file_dir='./temp/ill_test/' #directory where files for corr_pc will be placed. I recommend using full path rather than relative path as here.
file_name='ill_test'  #prefix to be appended to the files being saved. Should be different for different datasets.

os.makedirs(file_dir,exist_ok=True)

In [21]:
""" this function generates the input file for corr_pc
 corr_pc takes two samples to compute cross correlations. 
 Shape sample: Sample of galaxies with shapes. For density correlations it is not necessary to pass shapes. 
               Prefix of filenames for this sample should be passed vias file_nameS
 Density sample: Sample of galaxies/particles that trace density field. For density correlations it is not necessary to pass shapes. 
                 If shapes are passed for this sample, w++ can be computed. Prefix of filenames for this sample should be passed vias file_nameD.
                 If auto correlation is to be computed, simply set file_nameD=file_nameS.
 file_name: This is the prefix corr_pc will use to save the output files containing two point correlations. Input file to be passed to corr_pc is 
            also saved with this name. input file is saved in file_dir and the outputs are saved in new directory, file_dir+corr_data_out.
 corr: two point correlation to be computed. This should be density-density (wgg,wmm), shape-density (wg+) or shape-shape(w++). 
        When computing wg+, corr_pc will also compute and save wgg by default since pair counting is done anyways. 
        When doing w++, wg+ and wgg are also computed and saved.
 n_shape, n_density: number of galaxies in the shape sample and density sample respectively.
 do_jk: Whether to do jackknifing or not.
 njk: number of jackknife regions. jackknife regions are assumed to be labelled from 0...njk-1.
"""
def inp_gen(file_dir,file_name,file_nameS,file_nameD,corr=1,n_shape=0,n_density=0,njk=100,do_jk=1):
    s=""
    s=s+"which_corr     "+str(corr)+"\n" #0: density-density, 1: ia: shape-density, 2 ia:shape-shape,
    s=s+"coordinates    6"+"\n"  #0: rp-pi
    s=s+"estimator      0"+"\n"  #0:landy-szalay (LS): cross. works for both auto and cross correlations. For auto, pass same filenames for shape and density samples 
                                 # 1: LS:auto, for auto correlation only
    s=s+"data_sorted     1"+"\n"  #0: no sorting, 1:sorted by z, 2:sorted by dec (y)
    s=s+"use_comoving    1"+"\n"
    s=s+"do_jk           "+str(do_jk)+"\n" #0: no jackknife, 1: do jackknife. jk regions must be passed in a file. are assumed to be from 0..njk-1
    s=s+"sig_crit        0"+"\n" #for weak lensing, not important for ia, gg corrs.

    #following passes information about shape sample
    s=s+"shape_pos       "+file_dir+file_nameS+'_pos.dat'+"\n"  # x, y coordinates
    s=s+"shape_z         "+file_dir+file_nameS+'_z.dat'+"\n"    # z coordinate
    s=s+"shape_e         "+file_dir+file_nameS+'_e.dat'+"\n"    # e1, e2
    s=s+"shape_wt         "+file_dir+file_nameS+'_wt.dat'+"\n"  # weights on galaxies. If no weighting, pass 0
#     s=s+"shape_wt         0"+"\n"
    s=s+"shape_jk        "+file_dir+file_nameS+'_jk.dat'+"\n"   # jackknife regions

    # following passes information about density sample
    s=s+"density_patch   0"+"\n" #for weak lensing, not important for ia, gg corrs.
    s=s+"density_pos     "+file_dir+file_nameD+'_pos.dat'+"\n"  # x, y coordinates
    s=s+"density_z       "+file_dir+file_nameD+'_z.dat'+"\n"    # z coordinate
    s=s+"density_wt       "+file_dir+file_nameD+'_wt.dat'+"\n"  # weights on galaxies/particles. If no weighting, pass 0
    s=s+"density_jk      "+file_dir+file_nameD+'_jk.dat'+"\n"   # jackknife regions    
    s=s+"density_e      0"+"\n"   #if doing shape-shape, this file should be passed. # e1, e2

    # following passes information about randoms corresponding to shape sample
    s=s+"Srandom_pos      0"+"\n"
    s=s+"Srandom_z        0"+"\n"
    s=s+"Srandoms_wt      0"+"\n"
    s=s+"Srand_jk         0"+"\n"

    # following passes information about randoms corresponding to density sample
    s=s+"Drandom_patch    0"+"\n"
    s=s+"Drandom_pos      0"+"\n"
    s=s+"Drandom_z        0"+"\n"
    s=s+"Drandoms_wt      0"+"\n"
    s=s+"drand_jk         0"+"\n"

    s=s+"distances        0"+"\n"  #not needed for PB
    s=s+"patch_file       0"+"\n"  #not needed for PB
    s=s+"out_file         "+file_dir+"corr_data_out/"+file_name+"_"+"\n"  # prefix for the output files.
    
    os.makedirs(file_dir+"corr_data_out/",exist_ok=True) #directory where outputs are saved.
    
    s=s+"n_threads       0"+"\n"   # number of openmp threads to use. 0== use maximum available.
    s=s+"n_shape         "+str(n_shape)+"\n"  #number of galaxies in shape sample
    s=s+"n_density       "+str(n_density)+"\n"  #number of galaxies in density sample
    s=s+"n_Srand         0"+"\n"
    s=s+"n_Drand         0"+"\n"
    s=s+"rand_subsample  0"+"\n"  #subsample randoms to speed up calculations.
    s=s+"n_jk          "+str(njk)+"\n"  #number of jk regions. regions are assumed to be labelled from 0....njk-1
    s=s+"n_patch         0 "+"\n"#not needed for PB

    s=s+"binR_min        0.1"+"\n"  # rp_min for the bins
    s=s+"binR_max       10"+"\n" # rp_max for the bins
    s=s+"n_bins         10"+"\n" #number of rp bins to use
    s=s+"lin_bin         0"+"\n"  #linear or log bins. 0: log, 1:linear
    s=s+"n_p_bin         200"+"\n"  #number of pi bins to use
    s=s+"pmin         -100"+"\n"  # pi_min (line of sight or z separation)
    s=s+"pmax         100"+"\n"   # pi_max (line of sight or z separation)
    s=s+"z_min         0"+"\n"    
    s=s+"z_max         300"+"\n" 
    s=s+"dz            0.0001"+"\n" #not needed for PB
    s=s+"z_sep_min     -100"+"\n"  
    s=s+"z_sep_max     100"+"\n"   
    s=s+"periodic_box  1"+"\n"
    s=s+"box_size     205"+"\n"  #size of the box.

    fil=open(file_dir+file_name+'.inp','w')
    fil.write(s)


In [5]:
"""
This function splits the sim box into a grid in x-y plane. Number of grid splits along each axis are given by njk1, for a total of njk1^2 sub-sample regions.
After this, each point in data (galaxy or particle) is assigned to its sub-sample.
"""
def jk_box(data=[],Lbox=None,njk1=7):
    if Lbox is None:
        Lbox=max(data['x'])
    xjk=np.linspace(0,Lbox,njk1+1)
    yjk=np.linspace(0,Lbox,njk1+1)
    jk=np.zeros(len(data))
    jjk=0
    for i in np.arange(njk1):
        xil=xjk[i]
        xih=xjk[i+1]
        x1=data['x']>xil
        x2=data['x']<xih
        for j in np.arange(njk1):
            yil=yjk[j]
            yih=yjk[j+1]
            y1=data['y']>yil
            y2=data['y']<=yih
            jk[x1*x2*y1*y2]=jjk
            jjk+=1
#     jk=np.array(jk,dtype=[('jk','int32')])
    return jk


In [6]:
ill_dir='/hildafs/datasets/IllustrisTNG/TNG300-1/TNG300-1_particle_data/'

In [7]:
S_dat_name='1mil_star'
D_dat_name='1mil_gas'

In [8]:
#read in shape sample
fname=ill_dir+S_dat_name+'_coords.p'
with open(fname, "rb") as input_file:
    S_dat = Table(pickle.load(input_file)/1000,names=('x','y','z'))
fname=ill_dir+S_dat_name+'_masses.p'
with open(fname, "rb") as input_file:
    S_dat['mass'] = pickle.load(input_file)
S_dat['jk'] = jk_box(data=S_dat)
S_dat.sort('z')
# S_dat['e1']=
# S_dat['e2']=
S_dat

x,y,z,mass,jk
float64,float64,float64,float32,float64
184.97142509334796,82.36620462289149,0.0004184300757106685,0.00061077875,44.0
98.15542604033374,76.77960459808783,0.0004625828991211765,0.00044722008,23.0
29.724523039923643,133.42351390988458,0.0016041818459564588,0.00033856765,11.0
138.46974656755452,64.45575552376482,0.001645282847858534,0.00039742608,30.0
29.723419320332464,133.41542865155202,0.001969382922945162,0.00038760502,11.0
98.15897718747019,76.76825714701859,0.0021171065981962405,0.0005347482,23.0
98.15486691845335,76.76772026044443,0.0021292246548752385,0.0005509968,23.0
86.57732396677015,66.77505943632092,0.002448944916689726,0.00038565573,16.0
48.17413593858212,190.64708200998075,0.002596680741539132,0.00037900358,13.0


In [9]:
# read in density sample.
fname=ill_dir+D_dat_name+'_coords.p'
with open(fname, "rb") as input_file:
    D_dat = Table(pickle.load(input_file)/1000,names=('x','y','z'))
fname=ill_dir+D_dat_name+'_masses.p'
with open(fname, "rb") as input_file:
    D_dat['mass'] = pickle.load(input_file)
D_dat['jk'] = jk_box(data=D_dat)
D_dat.sort('z')

D_dat

x,y,z,mass,jk
float64,float64,float64,float32,float64
201.74382199482628,186.62446755558952,0.0010746587679719148,0.00042265223,48.0
112.03957380171954,133.99870675815566,0.001099151768957717,0.0009420891,25.0
75.31632237874992,115.09074921327925,0.0012527712770938157,0.0008836131,17.0
109.49981485021705,35.457368290047995,0.002066933027210227,0.00072004244,22.0
194.16895018996678,78.28696891093165,0.0028902545514723813,0.0010298735,44.0
117.45813481975708,0.4793332953732586,0.0029688639588465638,0.0005414264,28.0
194.9630292854405,34.77584206365508,0.0029820121002203355,0.00063969987,43.0
184.97121887594722,82.42676344555392,0.003324822899680307,0.00058180897,44.0
59.41124261868263,117.98134902266382,0.0037107428850160326,0.0010569126,18.0


In [10]:
'e1' in S_dat.columns

False

In [17]:
"""
Function to write data files in format that can be read by corr_pc
"""
def write_data(data,file_name,file_dir,wt_col=None):
    file_name=file_dir+file_name
    np.savetxt(file_name+'_pos.dat',np.column_stack((data['x'],data['y']))) #x,y position of galaxies
    np.savetxt(file_name+'_z.dat',data['z']) # z position of galaxies
    if 'jk' in data.columns:
        np.savetxt(file_name+'_jk.dat',data['jk'],fmt='%i') #jackknife is read as integer
    if 'e1' in data.columns: #ellipticities. corr_pc takes e1, e2
        np.savetxt(file_name+'_e.dat',np.column_stack((data['e1'],data['e2'])))
    if wt_col is not None: #wt to be applied to each galaxy/particle.
        wt=data[wt_col]*1.
        wt/=wt.mean()
        np.savetxt(file_name+'_wt.dat',wt)

In [18]:
file_nameS=file_name+'_shape'
write_data(data=S_dat,file_name=file_nameS,file_dir=file_dir,wt_col='mass')

file_nameD=file_name+'_density'
write_data(data=D_dat,file_name=file_nameD,file_dir=file_dir,wt_col='mass')


In [19]:
S_dat['jk'].max()

48.0

In [22]:
inp_gen(file_dir=file_dir,file_name=file_name,file_nameS=file_nameS,file_nameD=file_nameD,corr=0,n_shape=len(S_dat),n_density=len(D_dat),
        njk=int(S_dat['jk'].max()+1),do_jk=1) 
#corr=0 will only do wgg like correlation, no shapes. set corr=1 for wg+ and wgg

In [ ]:
_=hist(S_dat['jk'],int(S_dat['jk'].max()+1))